In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm

DATA_DIR = "../data/"

In [2]:
data_tim = pd.read_csv(DATA_DIR+"Senior_TIM_v1.txt", sep="|", encoding='latin1')
data_bus_routes = pd.read_csv(DATA_DIR+"BusRoutes.txt", sep="|", encoding='latin1')

In [3]:
data_bus_routes.head()

,IDRoute,linkid
0,1,537277300
1,1,537277673
2,1,537277754
3,1,537277777
4,1,537277795


In [4]:
data_tim.head()

,linkid,Region_of_Origin,District_of_Origin,County_of_Origin,Average_Daily_SeniorPopulation_Travelling
0,80216810,R1 - AM Lisboa,Lisboa,Lisboa,2013.681187
1,80216810,R1 - AM Lisboa,Setúbal,Palmela,7688.296396
2,80216810,R1 - AM Lisboa,Lisboa,Oeiras,220.647248
3,80216810,R1 - AM Lisboa,Lisboa,Vila Franca de Xira,233.203075
4,80216812,R1 - AM Lisboa,Lisboa,Lisboa,261.606014


In [5]:
all_link_id = set(data_tim["linkid"]) | set(data_bus_routes["linkid"])
len(all_link_id)

84991

In [6]:
from herepy import GeocoderApi

geocoder_api = GeocoderApi(api_key="HhEntMfQMu0Dfq6xwWYZGCAEmIWXM1YUdPneSImQCD4")

# geocodes given search text
response = geocoder_api.free_form("80216810")
print(response.as_dict())

ModuleNotFoundError: No module named 'herepy'

In [7]:
response = geocoder_api.free_form("80216810")
print(response.as_dict())

NameError: name 'geocoder_api' is not defined

## Make post request and parse response

In [8]:
API_KEY = "xEm8G0fGfrsDHyEkeLVD3VtjIKXSFQSgDYw82Wm_q8Y"
BASE_URL = "https://route.ls.hereapi.com/routing/7.2/calculateroute.json"

linkid_info_df = pd.DataFrame()
failed_link_id = []
print("Translating {} links".format(len(all_link_id)))

linkid_it = iter(all_link_id)
for tmp_linkid in tqdm(linkid_it):

    first_linkid = tmp_linkid
    second_linkid = next(linkid_it)
    
    link_init = "link!-"+str(first_linkid) # 721130424
    link_end = "link!-"+str(second_linkid) # 721130424

    params = {"apiKey": API_KEY,
                "waypoint0": link_init,
                "waypoint1": link_end,
                "mode": "fastest;car;traffic:disabled"}
    try:
        r = requests.get(BASE_URL, params=params)
        json_response = r.json()
        org_dest_list = json_response["response"]["route"][0]["waypoint"]
    except KeyError:
        print("Failed gettin linkids", first_linkid, second_linkid)
        failed_link_id.append(first_linkid)
        failed_link_id.append(second_linkid)
        continue
    
    tmp_df = pd.DataFrame()
    tmp_df["linkid"] = [str(first_linkid), str(second_linkid)]
    tmp_df["latitude"] = [org_dest_list[0]["mappedPosition"]["latitude"], org_dest_list[1]["mappedPosition"]["latitude"]]
    tmp_df["longitude"] = [org_dest_list[0]["mappedPosition"]["longitude"], org_dest_list[1]["mappedPosition"]["longitude"]]
    tmp_df["road_name"] = [org_dest_list[0]["mappedRoadName"], org_dest_list[1]["mappedRoadName"]]
    linkid_info_df = pd.concat([linkid_info_df, tmp_df])

linkid_info_df = linkid_info_df.reset_index(drop=True)

0it [00:00, ?it/s]

Translating 84991 links


63it [00:17,  3.72it/s]

Failed gettin linkids 576717032 575144173


118it [00:32,  3.66it/s]

...

Failed gettin linkids 945290923 945290924


42459it [3:49:59,  3.12it/s]

Failed gettin linkids 867696371 867696372


42495it [3:50:10,  3.08it/s]


StopIteration: 

In [10]:
failed_link_id

[576717032,
 575144173,
 813433231,
 924320142,
 576717980,
 576717997,
 762578126,
 762578127,
 762578140,
 820511965,
 1157891790,
 1157891791,
 576718562,
 575145699,
 576719516,
 576719517,
 575146847,
 576719714,
 576719816,
 576719817,
 576720069,
 576720070,
 576720939,
 1220022318,
 576721309,
 913052062,
 819466760,
 587731465,
 575148558,
 575148559,
 576721698,
 1212683041,
 1181226548,
 575149622,
 576722677,
 575149814,
 812128369,
 576723055,
 576723456,
 575150593,
 1213471244,
 575150605,
 575151277,
 576724139,
 575151293,
 575151294,
 916200823,
 576724345,
 575151588,
 762584551,
 767303317,
 767303321,
 1212948544,
 1212948545,
 542385253,
 542385254,
 1181230526,
 575153600,
 1217930726,
 542385641,
 911746957,
 542386071,
 542386994,
 542386995,
 542387011,
 915155780,
 542387041,
 542387045,
 812030028,
 542387785,
 1054617554,
 1054617555,
 576731065,
 915158970,
 1162361379,
 941111846,
 576732453,
 915160355,
 576732838,
 1160265382,
 1166032550,
 1160265381,


In [9]:
linkid_info_df

,linkid,latitude,longitude,road_name
0,575143936,41.174556,-8.683999,Rua Manuel Dias da Fonseca
1,575143938,41.185590,-8.691142,Rua de Álvaro Castelões
0,575143939,41.185130,-8.690613,Rua de Álvaro Castelões
1,575143940,41.185312,-8.690828,Rua de Álvaro Castelões
0,576716805,41.239055,-8.690425,Rua Vilar do Senhor
...,...,...,...,...
1,587726830,38.649399,-9.044857,Rua da Liberdade
0,587726831,38.649777,-9.045068,Rua da Liberdade
1,1171259379,41.155316,-8.591116,Praça das Flores
0,1171259380,41.155482,-8.591116,Praça das Flores


In [11]:
linkid_info_df.to_csv(DATA_DIR+"link_id_dictionary.csv", index=False)

In [13]:
pd.DataFrame(failed_link_id, columns=["linkid"]).to_csv(DATA_DIR+"link_id_missing", index=False)

,linkid
0,576717032
1,575144173
2,813433231
3,924320142
4,576717980
...,...
1127,746847309
1128,945290923
1129,945290924
1130,867696371


### Get link_id info that failed to obtain on the first step

In [15]:
failed_link_id = pd.read_csv(DATA_DIR+"link_id_missing.csv")

In [16]:
failed_link_id["linkid"].values

array([576717032, 575144173, 813433231, ..., 945290924, 867696371,
       867696372])

In [20]:
API_KEY = "xEm8G0fGfrsDHyEkeLVD3VtjIKXSFQSgDYw82Wm_q8Y"
BASE_URL = "https://route.ls.hereapi.com/routing/7.2/calculateroute.json"

remaining_link_id = pd.DataFrame()
invalid_link = []

print("Translating {} links".format(len(failed_link_id["linkid"].values)))

for tmp_linkid in tqdm(failed_link_id["linkid"].values):
    
    link_init = "link!-"+str(tmp_linkid) # 721130424
    link_end = "link!-721130424"

    params = {"apiKey": API_KEY,
                "waypoint0": link_init,
                "waypoint1": link_end,
                "mode": "fastest;car;traffic:disabled"}
    try:
        r = requests.get(BASE_URL, params=params)
        json_response = r.json()
        org_dest_list = json_response["response"]["route"][0]["waypoint"]
    except KeyError:
        #print("Failed getting link", tmp_linkid)
        invalid_link.append(tmp_linkid)
        continue
    
    tmp_df = pd.DataFrame()
    tmp_df["linkid"] = [str(tmp_linkid)]
    tmp_df["latitude"] = [org_dest_list[0]["mappedPosition"]["latitude"]]
    tmp_df["longitude"] = [org_dest_list[0]["mappedPosition"]["longitude"]]
    tmp_df["road_name"] = [org_dest_list[0]["mappedRoadName"]]
    remaining_link_id = pd.concat([remaining_link_id, tmp_df])

remaining_link_id = remaining_link_id.reset_index(drop=True)

  0%|          | 0/1132 [00:00<?, ?it/s]

Translating 1132 links


100%|██████████| 1132/1132 [05:43<00:00,  3.30it/s]


In [21]:
remaining_link_id.to_csv(DATA_DIR+"link_id_dictionary_pt2.csv", index=False)
pd.DataFrame(invalid_link, columns=["linkid"]).to_csv(DATA_DIR+"link_id_invalid.csv", index=False)
print("Link_ids with no mapping: ", len(invalid_link))

Link_ids with no mapping:  499


### Merge on final translation table

In [24]:
link_id_dict_pt1 = pd.read_csv(DATA_DIR+"link_id_dictionary_pt1.csv")
link_id_dict_pt2 = pd.read_csv(DATA_DIR+"link_id_dictionary_pt2.csv")
link_id_all = pd.concat([link_id_dict_pt1, link_id_dict_pt2]).reset_index()
link_id_all = link_id_all.drop(["index"], axis=1)
link_id_all.to_csv(DATA_DIR+"link_id_dictionary_final.csv", index=False)

In [25]:
link_id_all

,linkid,latitude,longitude,road_name
0,575143936,41.174556,-8.683999,Rua Manuel Dias da Fonseca
1,575143938,41.185590,-8.691142,Rua de Álvaro Castelões
2,575143939,41.185130,-8.690613,Rua de Álvaro Castelões
3,575143940,41.185312,-8.690828,Rua de Álvaro Castelões
4,576716805,41.239055,-8.690425,Rua Vilar do Senhor
...,...,...,...,...
84486,920645259,38.505458,-8.891222,Setúbal-Tróia
84487,867433175,41.169947,-8.621508,Rua de Monsanto
84488,746847309,38.778340,-9.097531,Avenida Dom João II
84489,945290923,40.843855,-8.390797,Avenida do Milénio
